# Example of using SpecialtyInsurance Simulator


In [1]:
### Set Simulation Parameters
import os
from logger.arguments import get_arguments

# Read arguments from logger.arguments
sim_args, manager_args, broker_args, syndicate_args, reinsurancefirm_args, shareholder_args, risk_args = get_arguments()

# Reset arguments
sim_args["max_time"] = 300   # Simulation time span unit day
manager_args["lead_top_k"] = 2   # Number of syndicates competing for the lead quote
manager_args["follow_top_k"] = 1   # Number of syndicates following the lead strategy
broker_args["num_brokers"] = 1   # Number of brokers in the insurance market
syndicate_args["num_syndicates"] = 3   # Number of syndicates in the insurance market
syndicate_args["lead_line_size"] = 0.90  # The percentage of risk covered by lead syndicate
syndicate_args["follow_line_size"] = 0.10  # The percentage of risk covered by follow syndicate
shareholder_args["num_shareholders"] = 1   # Number of shareholders in the insurance market
risk_args["num_risks"] = 1  # Number of risk categories
risk_args["num_categories"] = 4  # Number of risk categories

# No reinsurance mechanism included in this stage
with_reinsurance = False   

# Load one risk model to all syndicates
num_risk_models = 1   

In [2]:
# Generate Catastrophes
from environment.risk_generator import RiskGenerator

# Create catastrophe list and catastrophe configurations
catastrophes, risk_model_configs = RiskGenerator(num_risk_models, sim_args, risk_args).generate_risks()
print(catastrophes[0].get("risk_start_time"))

11


In [3]:
# Generate Insurance Market
from environment.market_generator import MarketGenerator

# Create lists of brokers, syndicates, reinsurancefirms, and shareholders
brokers, syndicates, reinsurancefirms, shareholders = MarketGenerator(with_reinsurance, 
                                                                      num_risk_models, 
                                                                      sim_args, 
                                                                      broker_args, 
                                                                      syndicate_args, 
                                                                      reinsurancefirm_args, 
                                                                      shareholder_args, 
                                                                      risk_model_configs).generate_agents()

In [5]:
# Input risk from broker

# Risk brought by broker: time to the market and number of risks

for i in range(len(brokers)):
    risks = brokers[i].generate_risks(catastrophes)
    print(risks)

[{'risk_id': 0, 'broker_id': '0', 'risk_start_time': 11, 'risk_end_time': 371, 'risk_factor': 0.5, 'risk_category': 4, 'risk_value': 1000.0}]


In [6]:
# Run the simulation
from manager.ai_model.runner import AIRunner
from manager.game_model.runner import GameRunner

model = 0
if model == 0: 
    runner = AIRunner(sim_args, manager_args, brokers, syndicates, reinsurancefirms, shareholders, catastrophes, risk_model_configs, with_reinsurance, num_risk_models)
elif model == 1:
    runner = GameRunner(sim_args, manager_args, brokers, syndicates, reinsurancefirms, shareholders, catastrophes, risk_model_configs, with_reinsurance, num_risk_models)
runner.run()

/opt/homebrew/lib/python3.8/site-packages/ray/rllib/algorithms/algorithm.py:483: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/opt/homebrew/lib/python3.8/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/opt/homebrew/lib/python3.8/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could 

(RolloutWorker pid=1816) Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=1816, ip=127.0.0.1, actor_id=0dadafe8d563d9b1fc6a1c0f01000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x14f021be0>)
(RolloutWorker pid=1816)   File "/Users/yubi/Documents/KCLFirstYear/Simulator-SpecialtyInsurance/environment/environment.py", line 160, in step
(RolloutWorker pid=1816)     market = self.mm.evolve(self.dt)
(RolloutWorker pid=1816)   File "/Users/yubi/Documents/KCLFirstYear/Simulator-SpecialtyInsurance/manager/market_manager.py", line 323, in evolve
(RolloutWorker pid=1816)     self.market.time = start_time
(RolloutWorker pid=1816) AttributeError: 'NoneType' object has no attribute 'time'
(RolloutWorker pid=1816) 
(RolloutWorker pid=1816) The above exception was the direct cause of the following exception:
(RolloutWorker pid=1816) 
(RolloutWorker pid=1816) ray::RolloutWorker.__init__() (pid

ValueError: Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.8/site-packages/ray/rllib/utils/pre_checks/env.py", line 363, in check_multiagent_environments
    results = env.step(sampled_action)
  File "/Users/yubi/Documents/KCLFirstYear/Simulator-SpecialtyInsurance/environment/environment.py", line 160, in step
    market = self.mm.evolve(self.dt)
  File "/Users/yubi/Documents/KCLFirstYear/Simulator-SpecialtyInsurance/manager/market_manager.py", line 323, in evolve
    self.market.time = start_time
AttributeError: 'NoneType' object has no attribute 'time'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.8/site-packages/ray/rllib/utils/pre_checks/env.py", line 81, in check_env
    check_multiagent_environments(env)
  File "/opt/homebrew/lib/python3.8/site-packages/ray/rllib/utils/pre_checks/env.py", line 368, in check_multiagent_environments
    raise ValueError(
ValueError: Y

In [ ]:
from environment.event.add_catastrophe import AddCatastropheEvent
from environment.event.add_attritionalloss import AddAttritionalLossEvent
from environment.event.add_risk import AddRiskEvent
from environment.event.add_premium import AddPremiumEvent
from environment.event.add_claim import AddClaimEvent
from environment.event_generator import EventGenerator
from manager.event_handler import EventHandler
from environment.market import NoReinsurance_RiskOne
import numpy as np

catastrophe_events = EventGenerator(risk_model_configs).generate_catastrophe_events(catastrophes)
attritional_loss_events = EventGenerator(risk_model_configs).generate_attritional_loss_events(sim_args, catastrophes)
broker_risk_events = EventGenerator(risk_model_configs).generate_risk_events(brokers, catastrophes)
broker_premium_events = []
broker_claim_events = []
time = 0
market = NoReinsurance_RiskOne(time, sim_args["max_time"], manager_args, brokers, syndicates, 
                               shareholders, catastrophes, risk_model_configs, catastrophe_events, 
                               attritional_loss_events, broker_risk_events, broker_premium_events, 
                               broker_claim_events)
event_handler = EventHandler(sim_args["max_time"], catastrophe_events, attritional_loss_events, broker_risk_events, broker_premium_events, broker_claim_events)

step_time = 1
market_start_time = market.time
market_end_time = market.time + step_time

upcoming_catastrophe = [
            e.risk_id for e in event_handler.upcoming_catastrophe.values() if isinstance(e, AddCatastropheEvent)
        ]
upcoming_attritional_loss = [
            e.risk_id for e in event_handler.upcoming_attritional_loss.values() if isinstance(e, AddAttritionalLossEvent)
        ]
upcoming_broker_risk = [
            e.risk_id for e in event_handler.upcoming_broker_risk.values() if isinstance(e, AddRiskEvent)
        ]
upcoming_broker_premium = [
            e.risk_id for e in event_handler.upcoming_broker_premium.values() if isinstance(e, AddPremiumEvent)
        ]
upcoming_broker_claim = [
            e.risk_id for e in event_handler.upcoming_broker_claim.values() if isinstance(e, AddClaimEvent)
        ]
step_time = 1
market = event_handler.forward(market, step_time)

newly_added_catastrophe_events = {
            e.risk_id: e.risk_start_time
            for e in event_handler.completed_catastrophe.values()
            if isinstance(e, AddCatastropheEvent) and (e.risk_id in upcoming_catastrophe)
        }

newly_added_attritional_loss_events = {
            e.risk_id: e.risk_start_time
            for e in event_handler.completed_attritional_loss.values()
            if isinstance(e, AddAttritionalLossEvent) and (e.risk_id in upcoming_attritional_loss)
        }

newly_added_broker_risk_events = {
            e.risk_id: e.risk_start_time
            for e in event_handler.completed_broker_risk.values()
            if isinstance(e, AddRiskEvent) and (e.risk_id in upcoming_broker_risk)
        }

newly_added_broker_premium_events = {
            e.risk_id: e.risk_start_time
            for e in event_handler.completed_broker_premium.values()
            if isinstance(e, AddPremiumEvent) and (e.risk_id in upcoming_broker_premium)
        }

newly_added_broker_claim_events = {
            e.risk_id: e.risk_start_time
            for e in event_handler.completed_broker_claim.values()
            if isinstance(e, AddClaimEvent) and (e.risk_id in upcoming_broker_claim)
        }

catastrophe_event_start_times = np.array(
            [
                newly_added_catastrophe_events.get(risk_id)
                for risk_id in upcoming_catastrophe
                if newly_added_catastrophe_events.get(risk_id) != None
            ]
        )

attritional_loss_event_start_times = np.array(
            [
                newly_added_attritional_loss_events.get(risk_id)
                for risk_id in upcoming_attritional_loss
                if newly_added_attritional_loss_events.get(risk_id) != None
            ]
        )

broker_risk_event_start_times = np.array(
            [
                newly_added_broker_risk_events.get(risk_id)
                for risk_id in upcoming_broker_risk
                if newly_added_broker_risk_events.get(risk_id) != None
            ]
        )

broker_premium_event_start_times = np.array(
            [
                newly_added_broker_premium_events.get(risk_id)
                for risk_id in upcoming_broker_premium
                if newly_added_broker_premium_events.get(risk_id) != None
            ]
        )

broker_claim_event_start_times = np.array(
            [
                newly_added_broker_claim_events.get(risk_id)
                for risk_id in upcoming_broker_claim
                if newly_added_broker_claim_events.get(risk_id) != None
            ]
        )

event_start_times = np.concatenate((catastrophe_event_start_times,
                                    attritional_loss_event_start_times,
                                    broker_risk_event_start_times,
                                    broker_premium_event_start_times,
                                    broker_claim_event_start_times))

sorted_unique_start_times = np.sort(np.unique(event_start_times))

print(event_handler.completed_attritional_loss.values())